In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, random_split
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import seaborn as sns
import pickle
import os
from PIL import Image
import re
import random
import shutil
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
import os
from PIL import Image
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchvision.models import vgg16
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision.models import vgg16
from sklearn.metrics import accuracy_score, roc_curve, confusion_matrix
import matplotlib.pyplot as plt
import glob
import re
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler
from skimage.metrics import structural_similarity as ssim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
path = '/content/drive/MyDrive/DL_ASSIGN_04/Dataset_BUSI_with_GT'

In [ ]:
# # # IMPORTANT


# l = ['normal','malignant','benign']
# image_paths = []
# mask_paths = []
# labels  = []
# for root, dirs, files in os.walk(path):

#   for file in files:
#       mask = os.path.join(root, file)
#       if 'mask' in mask:
#         mask_paths.append(mask)
#         # print(mask)
#         x = re.split("\s", mask,1)
#         image = x[0] +' ' +(x[1].replace("_mask", ""))
#         image_paths.append(image)

# for i in image_paths:
#   for index, element in enumerate(l):
#     if element in i:
#       labels.append(index)
        # print(image)


In [ ]:
class Image_Dataset(Dataset):
      def __init__(self,root_dir,transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths, self.mask_paths , self.labels = self.data_path()

      def __len__(self):
        return len(self.mask_paths)

      def __getitem__(self,idx):
        image = self.image_paths[idx]
        mask = self.mask_paths[idx]
        image = Image.open(image).convert("L")
        mask = Image.open(mask).convert("L")
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
        label = self.labels[idx]
        return image , mask ,label

      def data_path(self):
        l = ['normal','malignant','benign']
        image_paths = []
        mask_paths = []
        labels = []
        for root, dirs, files in os.walk(path):

          for file in files:
              mask = os.path.join(root, file)
              if 'mask' in mask:
                mask_paths.append(mask)
                # print(mask)
                x = re.split("\s", mask,1)
                image = x[0] +' ' +(x[1].replace("_mask", ""))
                image_paths.append(image)
                # print(image)
        for i in image_paths:
          for index, element in enumerate(l):
            if element in mask:
              labels.append(index)
        return image_paths , mask_paths , labels




In [ ]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

In [ ]:
batch_size = 32

In [ ]:
dataset = Image_Dataset(root_dir=path, transform=transform)


In [ ]:
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

In [ ]:
train_data, val_data, test_data = torch.utils.data.random_split(
    dataset, [train_size, val_size, test_size]
)

In [ ]:
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle= True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [ ]:
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        # Encoder
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.layer_1 = nn.Sequential( nn.Conv2d(1, 64, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True),
                       nn.Conv2d(64, 64, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True))
        self.layer_2 = nn.Sequential( nn.Conv2d(64, 128, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True),
                       nn.Conv2d(128, 128, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True))
        self.layer_3 = nn.Sequential( nn.Conv2d(128, 256, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True),
                       nn.Conv2d(256, 256, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True))
        self.layer_4 = nn.Sequential( nn.Conv2d(256, 512, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True),
                       nn.Conv2d(512, 512, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True))
        self.layer_5 = nn.Sequential( nn.Conv2d(512, 1024, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True),
                       nn.Conv2d(1024, 1024, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True))

        #Decoder
        self.up_conv_1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.layer_6 = nn.Sequential(
                       nn.Conv2d(1024, 512, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True),
                       nn.Conv2d(512, 512, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True))
        self.up_conv_2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.layer_7 = nn.Sequential(
                       nn.Conv2d(512, 256, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True),
                       nn.Conv2d(256, 256, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True))
        self.up_conv_3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.layer_8 = nn.Sequential(
                       nn.Conv2d(256, 128, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True),
                       nn.Conv2d(128, 128, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True))
        self.up_conv_4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.layer_9 = nn.Sequential(
                       nn.Conv2d(128, 64, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True),
                       nn.Conv2d(64, 64, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True))
        self.layer_10 = nn.Conv2d(64, 1, kernel_size=1)

    def forward(self, x):
        # Encoder
        x1 = self.layer_1(x)  #...
        x2 = self.max_pool(x1)
        x3 = self.layer_2(x2) #...
        x4 = self.max_pool(x3)
        x5 = self.layer_3(x4) #...
        x6 = self.max_pool(x5)
        x7 = self.layer_4(x6) #...
        x8 = self.max_pool(x7)
        x9 = self.layer_5(x8)

        #Decoder

        x10 = self.up_conv_1(x9)
        size = (x10.size())[2]
        cropped_tensor = x7[:, :,:size, :size]
        x11 = self.layer_6(torch.cat([cropped_tensor, x10], dim=1))
        # print(x11.size())


        x12 = self.up_conv_2(x11)
        size = (x12.size())[2]
        cropped_tensor = x5[:, :,:size, :size]
        x13 = self.layer_7(torch.cat([cropped_tensor, x12], dim=1))


        x14 = self.up_conv_3(x13)
        size = (x14.size())[2]
        cropped_tensor = x3[:, :,:size, :size]
        x15 = self.layer_8(torch.cat([cropped_tensor, x14], dim=1))


        x16 = self.up_conv_4(x15)
        size = (x16.size())[2]
        cropped_tensor = x1[:, :,:size, :size]
        x17 = self.layer_9(torch.cat([cropped_tensor, x16], dim=1))
        x18 = self.layer_10(x17)
        # print(x18.size())
        return x18


if __name__ == "__main__":
  model = UNet()


In [ ]:
def  testdice_loss(y_true, y_pred, smooth=1e-6):
    y_true_flat = y_true.view(-1)
    y_pred_flat = y_pred.view(-1)

    y_true_flat_numpy = y_true_flat.numpy().tolist()
    y_pred_flat_numpy = y_pred_flat.numpy().tolist()

    intersection = np.sum(np.array(y_true_flat_numpy) * np.array(y_pred_flat_numpy))
    union = np.sum(np.array(y_true_flat_numpy)) + np.sum(np.array(y_pred_flat_numpy))

    dice = 1-((2. * intersection + smooth) / (union + smooth))
    return dice
def testdice_score(y_true, y_pred, smooth=1e-6):
    y_true_flat = y_true.view(-1)
    y_pred_flat = y_pred.view(-1)

    y_true_flat_numpy = y_true_flat.numpy().tolist()
    y_pred_flat_numpy = y_pred_flat.numpy().tolist()

    intersection = np.sum(np.array(y_true_flat_numpy) * np.array(y_pred_flat_numpy))
    union = np.sum(np.array(y_true_flat_numpy)) + np.sum(np.array(y_pred_flat_numpy))

    dice = (2. * intersection + smooth) / (union + smooth)
    return dice

optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
patience = 3

In [ ]:
# for epoch in range(1):
#     model.train()  # Set the model to training mode
#     running_loss = 0.0
#     train_true = []
#     train_pred = []
#     for images, masks, class_ in train_loader:
#         images = images.to(device)  # Move data to GPU if available
#         masks = masks.to(device)
#         print(masks)
#         print(masks.size())
#         optimizer.zero_grad()  # Zero the parameter gradients\
#         y_true_flat = masks.view(-1)
#         y_pred_flat = y_pred.view(-1)

#         y_true_flat_numpy = y_true_flat.numpy().tolist()
#         y_pred_flat_numpy = y_pred_flat.numpy().tolist()


#         # Forward pass
#         outputs = model(images)
#         print(outputs)
#         print(outputs.size())
#         loss = dice_loss(masks,outputs)
#         dice = dice_score(masks,outputs)
#         print(loss)
#         print(dice)
#         # Backward pass and optimize
#         loss.backward()
#         optimizer.step()
#         loss_train = loss.item()
#         dice_train = dice.item()
#         print(loss_train,dice_train)
#         running_loss += loss.item()
#         ground_truth = torch.cat(masks, dim=0).cpu().numpy()
#         predicted_segmentation = torch.cat(outputs, dim=0).cpu().numpy()
#         print(ground_truth,predicted_segmentation)
#     # Calculate average loss for the epoch
#     average_loss = running_loss / len(train_loader)
#     accuracy = accuracy_score(all_true_labels, all_predicted_labels)

#     print(f'Epoch [{epoch + 1}/{10}], Train Loss: {average_loss:.4f}')

# print('Finished Training')

In [ ]:
def dice_loss(y_true, y_pred, smooth=1e-6):
    intersection = torch.sum(y_true * y_pred)
    union = torch.sum(y_true) + torch.sum(y_pred)
    dice = (2.0 * intersection + smooth) / (union + smooth)
    loss = 1.0 - dice
    return loss
def dice_score(y_true, y_pred, smooth=1e-6):
    intersection = torch.sum(y_true * y_pred)
    union = torch.sum(y_true) + torch.sum(y_pred)
    dice = (2.0 * intersection + smooth) / (union + smooth)
    loss = 1.0 - dice
    return dice

In [ ]:
num_epochs = 1

In [ ]:
training_loss = []
validation_loss = []
training_dice_scores = []
validation_dice_scores = []
train_acc = []
val_acc = []
best_metric_value = float('-inf')
best_model_weights = None
for epoch in range(num_epochs):
    train_true = []
    train_pred = []
    val_true = []
    val_pred = []
    model.train()
    training_running_loss = 0.0
    val_running_loss = 0.0
    dice_train_running = 0.0
    dice_val_running = 0.0
    for images, masks, class_ in train_loader:
        images = images
        masks = masks

        optimizer.zero_grad()
        outputs = model(images)

        loss = dice_loss(masks,outputs)
        dice = dice_score(masks,outputs)
        loss.backward()
        optimizer.step()

        loss_train = loss.item()
        dice_train = dice.item()
        train_loss = training_running_loss + loss_train
        train_dice_score = dice_train_running + dice_train

        for i in range(len(images)):
          image_true = masks[i:i+1]
          image_pred = outputs[i:i+1]

          y_true_flat = image_true.view(-1)
          y_true_flat_numpy = y_true_flat.numpy().tolist()
          y_pred_flat = image_pred.view(-1)
          y_pred_flat_numpy = y_pred_flat.detach().numpy().tolist()
          train_pred.append(y_pred_flat_numpy)
          train_true.append(y_true_flat_numpy)
          print(len(train_true),len(train_pred))


    print(train_true[:400])
    print(train_pred[:400])

    average_train_loss = train_loss / len(train_loader)
    average_dice_score = train_dice_score /len(train_loader)
    training_loss.append(average_train_loss)
    training_dice_scores.append(average_dice_score)

    train_true_arr = np.array(train_true)
    train_pred_arr = np.array(train_pred)
    train_true_int = train_true_arr.astype(int)
    train_pred_int = train_pred_arr.astype(int)
    accuracy = accuracy_score(train_true_int, train_pred_int)
    train_acc.append(accuracy)
    print(f'Epoch [{epoch + 1}/{num_epochs}],  Train Loss: {average_train_loss:.4f}, Training Accuracy: {accuracy}')

    model.eval()
    with torch.no_grad():
      for images, masks, class_ in val_loader:
        images = images
        masks = masks

        optimizer.zero_grad()
        outputs = model(images)

        loss = dice_loss(masks,outputs)
        dice = dice_score(masks,outputs)
        loss_val = loss.item()
        dice_val = dice.item()
        val_loss = val_running_loss + loss_val
        val_dice_score = dice_val_running + dice_val

        for i in range(len(images)):
          image_true = masks[i:i+1]
          image_pred = outputs[i:i+1]

          y_true_flat = image_true.view(-1)
          y_true_flat_numpy = y_true_flat.numpy().tolist()
          y_pred_flat = image_pred.view(-1)
          y_pred_flat_numpy = y_pred_flat.detach().numpy().tolist()
          val_pred.append(y_pred_flat_numpy)
          val_true.append(y_true_flat_numpy)


      average_val_loss = val_loss/len(val_loader)
      average_val_dice_score = val_dice_score /len(val_loader)
      validation_loss.append(average_val_loss)
      validation_dice_scores.append(average_val_dice_score)




      scheduler.step(average_val_loss)

        # Early stopping check
      if average_val_loss < best_val_loss and average_train_loss < best_val_loss:
            best_val_loss = average_val_loss
            epoch = 0

      else:
          epoch += 1
          if epoch >= patience:
              print(f'Validation loss has not improved for {patience} consecutive epochs. Early stopping...')
      # Best Weigts
      if average_val_dice_score > best_metric_value and average_val_dice_score > 0.6:
        best_metric_value = average_val_dice_score
        best_model_weights = model.state_dict()
        best_model = model()
        torch.save(best_model_weights, 'best_model_weights.pth')
        torch.save(best_model, 'best_model.pth')\


    val_true_arr = np.array(val_true)
    val_pred_arr = np.array(val_pred)
    val_true_int = val_true_arr.astype(int)
    val_pred_int = val_pred_arr.astype(int)
    accuracy = accuracy_score(val_true_int, val_pred_int)
    val_acc.append(accuracy)

    print(f'Epoch [{epoch + 1}/{num_epochs}],  Validation Loss: {average_val_loss:.4f}, Validation Accuracy: {accuracy}')



x_axis = [i for i in range(len(training_loss))]
plt.plot(x_axis, training_loss, label='Train Loss')
plt.plot(x_axis, validation_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

x_axis = [i for i in range(len(train_acc))]
plt.plot(x_axis, train_acc, label='Train Accuracy')
plt.plot(x_axis, val_acc, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

x_axis = [i for i in range(len(training_dice_scores))]
plt.plot(x_axis, training_dice_scores, label='Train Loss')
plt.plot(x_axis, validation_dice_scores, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Dice Scores')
plt.legend()
plt.title('Training and Validation Dice_Scores')
plt.show()


1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 29
30 30
31 31
32 32
33 33
34 34
35 35
36 36
37 37
38 38
39 39
40 40
41 41
42 42
43 43
44 44
45 45
46 46
47 47
48 48
49 49
50 50
51 51
52 52
53 53
54 54
55 55
56 56
57 57
58 58
59 59
60 60
61 61
62 62
63 63
64 64
65 65
66 66
67 67
68 68
69 69
70 70
71 71
72 72
73 73
74 74
75 75
76 76
77 77
78 78
79 79
80 80
81 81
82 82
83 83
84 84
85 85
86 86
87 87
88 88
89 89
90 90
91 91
92 92
93 93
94 94
95 95
96 96
97 97
98 98
99 99
100 100
101 101
102 102
103 103
104 104
105 105
106 106
107 107
108 108
109 109
110 110
111 111
112 112
113 113
114 114
115 115
116 116
117 117
118 118
119 119
120 120
121 121
122 122
123 123
124 124
125 125
126 126
127 127
128 128
129 129
130 130
131 131
132 132
133 133
134 134
135 135
136 136
137 137
138 138
139 139
140 140
141 141
142 142
143 143
144 144
145 145
146 146
147 147
148 148
149 149
150 150
151 151
152 152


In [ ]:
num_epochs = 5

In [ ]:
training_loss = []
validation_loss = []
training_dice_scores = []
validation_dice_scores = []
train_acc = []
val_acc = []
best_metric_value = float('-inf')
best_model_weights = None
for epoch in range(num_epochs):
    train_true = []
    train_pred = []
    val_true = []
    val_pred = []
    model.train()
    training_running_loss = 0.0
    val_running_loss = 0.0
    dice_train_running = 0.0
    dice_val_running = 0.0
    for images, masks, class_ in train_loader:
        images = images
        masks = masks

        optimizer.zero_grad()
        outputs = model(images)

        loss = dice_loss(masks,outputs)
        dice = dice_score(masks,outputs)
        loss.backward()
        optimizer.step()

        loss_train = loss.item()
        dice_train = dice.item()
        train_loss = training_running_loss + loss_train
        train_dice_score = dice_train_running + dice_train

        y_pred_flat = outputs.view(-1)
        y_pred_flat_numpy = y_pred_flat.detach().numpy().tolist()
        train_pred.append(y_pred_flat_numpy)


        y_true_flat = masks.view(-1)
        y_true_flat_numpy = y_true_flat.numpy().tolist()
        train_true.append(y_true_flat_numpy)


    average_train_loss = train_loss / len(train_loader)
    average_dice_score = train_dice_score /len(train_loader)
    training_loss.append(average_train_loss)
    training_dice_scores.append(average_dice_score)

    accuracy = accuracy_score(train_true, train_pred)
    train_acc.append(accuracy)
    print(f'Epoch [{epoch + 1}/{num_epochs}],  Train Loss: {average_train_loss:.4f}, Training Accuracy: {accuracy}')
    print(train_true)
    print(train_pred)
    model.eval()
    with torch.no_grad():
      for images, masks, class_ in val_loader:
        images = images
        masks = masks

        optimizer.zero_grad()
        outputs = model(images)

        loss = dice_loss(masks,outputs)
        dice = dice_score(masks,outputs)
        loss_val = loss.item()
        dice_val = dice.item()
        val_loss = val_running_loss + loss_val
        val_dice_score = dice_val_running + dice_val

        image_ = outputs
        y_pred_flat = image_.view(-1)
        y_pred_flat_numpy = y_pred_flat.detach().numpy().tolist()
        val_pred.append(y_pred_flat_numpy)

      average_val_loss = val_loss/len(val_loader)
      average_val_dice_score = val_dice_score /len(val_loader)
      validation_loss.append(average_val_loss)
      validation_dice_scores.append(average_val_dice_score)

      y_true_flat = masks.view(-1)
      y_true_flat_numpy = y_true_flat.numpy().tolist()
      val_true.append(y_true_flat_numpy)


      scheduler.step(average_val_loss)

        # Early stopping check
      if average_val_loss < best_val_loss and average_train_loss < best_val_loss:
            best_val_loss = average_val_loss
            epoch = 0

      else:
          epoch += 1
          if epoch >= patience:
              print(f'Validation loss has not improved for {patience} consecutive epochs. Early stopping...')
      # Best Weigts
      if average_val_dice_score > best_metric_value and average_val_dice_score > 0.6:
        best_metric_value = average_val_dice_score
        best_model_weights = model.state_dict()
        best_model = model()
        torch.save(best_model_weights, 'best_model_weights.pth')
        torch.save(best_model, 'best_model.pth')
    accuracy = accuracy_score(val_true, val_pred)
    val_acc.append(accuracy)

    print(f'Epoch [{epoch + 1}/{num_epochs}],  Validation Loss: {average_val_loss:.4f}, Validation Accuracy: {accuracy}')



x_axis = [i for i in range(len(training_loss))]
plt.plot(x_axis, training_loss, label='Train Loss')
plt.plot(x_axis, validation_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

x_axis = [i for i in range(len(train_acc))]
plt.plot(x_axis, train_acc, label='Train Accuracy')
plt.plot(x_axis, val_acc, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

x_axis = [i for i in range(len(training_dice_scores))]
plt.plot(x_axis, training_dice_scores, label='Train Loss')
plt.plot(x_axis, validation_dice_scores, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Dice Scores')
plt.legend()
plt.title('Training and Validation Dice_Scores')
plt.show()


In [ ]:
test_true = []
test_pred = []
test_dice_score = []
test_running_loss = 0.0
test_dice_score_ = 0.0
model.eval()
with torch.no_grad():
  for images, masks, class_ in test_loader:
    images = images
    masks = masks
    optimizer.zero_grad()
    outputs = model(images)
    for i in range(len(images)):
      image_true = masks[i:i+1]
      image_pred = outputs[i:i+1]
      test_true.append(image_true)
      test_pred.append(image_pred)

      loss = testdice_loss(image_true, image_pred).item()
      dice = testdice_score(image_true, image_pred).item()
      test_dice_score.append(dice)

      test_dice_score_ += dice
      test_running_loss +=loss
  average_test_loss = test_running_loss/len(test_loader)
  average_test_dice = test_dice_score_/len(test_loader)
#loop
for i in range(5):
  if test_dice_score[i] >= 0.6:
    print(f'The Dice Score of the image is {test_dice_score[i]:.4f}')
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))


    axs[0].imshow(test_true[i])
    axs[0].set_title('Image')


    axs[1].imshow(test_pred[i])
    axs[1].set_title('Mask')


    plt.show()

  else:
    if test_dice_score[i] < 0.6:
      print(f'The Dice Score of the image is {test_dice_score[i]:.4f}')
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))


    axs[0].imshow(test_true[i])
    axs[0].set_title('Image')


    axs[1].imshow(test_pred[i])
    axs[1].set_title('Mask')


    plt.show()


In [ ]:
# print(test_true)
# print(test_pred)
# print(dice_score)

In [ ]:
#TSNE

In [ ]:
class encoder(nn.Module):
    def __init__(self):
        super(encoder, self).__init__()

        # Encoder
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.layer_1 = nn.Sequential( nn.Conv2d(1, 64, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True),
                       nn.Conv2d(64, 64, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True))
        self.layer_2 = nn.Sequential( nn.Conv2d(64, 128, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True),
                       nn.Conv2d(128, 128, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True))
        self.layer_3 = nn.Sequential( nn.Conv2d(128, 256, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True),
                       nn.Conv2d(256, 256, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True))
        self.layer_4 = nn.Sequential( nn.Conv2d(256, 512, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True),
                       nn.Conv2d(512, 512, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True))
        self.layer_5 = nn.Sequential( nn.Conv2d(512, 1024, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True),
                       nn.Conv2d(1024, 1024, kernel_size=3,padding = 1),
                       nn.ReLU(inplace=True))


    def forward(self, x):
          # Encoder
            x1 = self.layer_1(x)  #...
            # print(x1.size())
            x2 = self.max_pool(x1)
            # print(x2.size())
            x3 = self.layer_2(x2) #...
            # print(x3.size())
            x4 = self.max_pool(x3)
            # print(x4.size())
            x5 = self.layer_3(x4) #...
            # print(x5.size())
            x6 = self.max_pool(x5)
            # print(x6.size())
            x7 = self.layer_4(x6) #...
            # print(x7.size())
            x8 = self.max_pool(x7)
            # print(x8.size())
            x9 = self.layer_5(x8)
            # print(x9.size())

            return x9
if __name__ == "__main__":
    model_encoder = encoder()
    image = torch.rand(1, 1, 256, 256)

    print(model_encoder(image))


In [ ]:
image_ = []
label_ = [0,1,2,1,2,0,1,0,1,0]
for i in range(10):  # Loop from 0 to 9 (inclusive) for the first 10 items
    image, label,class_= dataset[i]

    print(class_)
    image_.append(image)

    # label_.append(class_)
images_np = np.array(image_)
labels_np = np.array(label_)
flattened_images = images_np.reshape(images_np.shape[0], -1)

tsne = TSNE(n_components=2, perplexity=5, random_state=0)
images = tsne.fit_transform(flattened_images)
plt.figure(figsize=(8, 6))
scatter = plt.scatter(images[:, 0], images[:, 1], c=labels_np, cmap='viridis')
plt.colorbar(scatter, label='Labels')
plt.title('TSNE Plot')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.show()

PART 2
**bold text**